In [ ]:
#  you need to install tensorflow-text,tf-models-official
!pip install -q tensorflow-text

In [ ]:
!pip install -q tf-models-official

In [ ]:
!pip install --q flair

In [ ]:
# if running from colabs, mount drive and upload the files in the google drive otherwise comment out
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import packages
import numpy as np 
import pandas as pd
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow import feature_column
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization 
tf.get_logger().setLevel('ERROR')
from matplotlib import pyplot
from flair.data import Sentence
from flair.embeddings import StackedEmbeddings, TransformerDocumentEmbeddings, DocumentPoolEmbeddings, FlairEmbeddings
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

In [ ]:
#upload dataset
filename =r'/Dataset/GermEval21_Toxic_/GermEval21_Toxic_.csv'
data=pd.read_csv(filename)

data.set_index('comment_id')
# df1 is the subtask1 for toxic
df1=data[['comment_text','Sub2_Engaging']]

In [ ]:
#Model one ANN- Split for training and validation and test set
# in future make a k-fold cross validation with sklearn
train, test = train_test_split(df1, test_size=0.2)
val, test = train_test_split(test, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

train_ds=train['comment_text'].astype(str).to_numpy()
val_ds=val['comment_text'].astype(str).to_numpy()
test_ds=test['comment_text'].astype(str).to_numpy()

whole_dataset=df1['comment_text'].astype(str).to_numpy()
whole_labels=df1['Sub2_Engaging'].astype(int).to_numpy()

train_labels=train['Sub2_Engaging'].astype(int).to_numpy()
val_labels=val['Sub2_Engaging'].astype(int).to_numpy()
test_labels=test['Sub2_Engaging'].astype(int).to_numpy()

In [ ]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((train_ds,train_labels))
valid_ds = tf.data.Dataset.from_tensor_slices((val_ds,val_labels))
# test_ds = tf.data.Dataset.from_tensor_slices((val_ds,val_labels))

whole_ds=tf.data.Dataset.from_tensor_slices((whole_dataset,whole_labels))

train_ds = train_ds.batch(BATCH_SIZE)
valid_ds = valid_ds.batch(BATCH_SIZE)

# PREFETCH

train_ds = train_ds.shuffle(5).prefetch(buffer_size=AUTOTUNE)
valid_ds = valid_ds.shuffle(5).prefetch(buffer_size=AUTOTUNE)

In [ ]:
#Model 1 Sentence_encoder
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer( "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2", name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2", name='multi_BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
classifier_model = build_classifier_model()
tf.keras.utils.plot_model(classifier_model)

loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = [tf.metrics.BinaryAccuracy()]

epochs = 50
steps_per_epoch =  tf.data.experimental.cardinality(train_ds).numpy()
# tf.data.experimental.cardinality(np.array(train_ds)).numpy()
# len(train)
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 1e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

# monitoring the val_loss is the best way as monitoring actual training loss results in overfitting ..
mcp_save = ModelCheckpoint('bert1.h5', save_best_only=True, monitor='val_loss', mode='min')

early_stop=EarlyStopping(monitor='val_loss', patience=3, verbose=0,restore_best_weights=True)

In [ ]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [ ]:
history = classifier_model.fit(train_ds,
                               validation_data=valid_ds,
                               epochs=epochs,callbacks=[mcp_save,early_stop], verbose=1)

In [ ]:
# plot loss during training
pyplot.subplot(211)
pyplot.title('Loss')
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='val')
pyplot.legend()
# plot accuracy during training
pyplot.subplot(212)
pyplot.title('Accuracy')
pyplot.plot(history.history['binary_accuracy'], label='train')
pyplot.plot(history.history['val_binary_accuracy'], label='val')
pyplot.legend()
pyplot.show()

In [ ]:
#model_1!!!!!

#load saved best weights to the model
classifier_model.load_weights('bert1.h5')
#load weights trained with gpu
# classifier_model.load_weights(r'/content/drive/MyDrive/Dataset/GermEval21_Toxic_/bert.h5')
# predict probabilities for test set and whole dataset
yhat_probs = classifier_model.predict(test['comment_text'], verbose=0)
yhat_classes=tf.cast(tf.round(tf.sigmoid(yhat_probs)), tf.int32)

yhat_df1probs = classifier_model.predict(df1['comment_text'], verbose=0)
yhat_df1classes=tf.cast(tf.round(tf.sigmoid(yhat_df1probs)), tf.int32)

df1['predictions_LaBse']=yhat_df1classes.numpy()



In [ ]:
#metrics for the small test set.....  5 excamples
# accuracy: (tp + tn) / (p + n)
print('Metrics for test set ONLY 5 examples\n')
accuracy = accuracy_score(test['Sub2_Engaging'], yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test['Sub2_Engaging'], yhat_classes,average='macro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test['Sub2_Engaging'], yhat_classes,average='macro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
# f1 = f1_score(df1['Sub1_Toxic'], df1['roberta_predictions'])
# print('F1 score: %f' % f1)

if precision+recall > 0:
  f1_score = 2*precision*recall/(precision+recall)
print('F1 score: %f' % f1_score)

In [ ]:
#metrics for Model 1--- sentence encoder!!!!!
# accuracy: (tp + tn) / (p + n)
print('Metrics for Setence _econder\n')
accuracy = accuracy_score(df1['Sub2_Engaging'],df1['predictions_LaBse'])
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(df1['Sub2_Engaging'], df1['predictions_LaBse'],average='macro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(df1['Sub2_Engaging'], df1['predictions_LaBse'],average='macro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
# f1 = f1_score(df1['Sub1_Toxic'], df1['roberta_predictions'])
# print('F1 score: %f' % f1)

if precision+recall > 0:
  f1_score = 2*precision*recall/(precision+recall)
print('F1 score: %f' % f1_score)

In [ ]:
#initialize transformer embeddings 
transformer_embedding = TransformerDocumentEmbeddings('xlm-roberta-base')  #xlm-large is too big for memory

#initialize flair embeddings
flair_embedding_forward = FlairEmbeddings('de-forward');
flair_embedding_backward = FlairEmbeddings('de-backward');
document_embeddings = DocumentPoolEmbeddings([flair_embedding_forward, flair_embedding_backward]) ;


In [ ]:
#Model 2-- flair embeddings
df1['flair_embeddings']= " "

#create features for tweets_training for Model 2 FLair embeddings
for i in range(0,len(df1)):

    # create a sentence
    sentence = Sentence(df1['comment_text'][i])

    # embed the sentence
    document_embeddings.embed(sentence)

    embedding = sentence.embedding.cpu()
    
    #save vector as numpy
    embedding = embedding.detach().numpy()
    
    #save vector as pandas dataframe
    embedding = pd.DataFrame(embedding)
    
    #make list out of sentence
    embedding = embedding[0].tolist()

    #add the embedding vector to the column of stacked embeddings
    df1['flair_embeddings'][i] = embedding

In [ ]:
#create features for tweets_training for Model 3

df1['roberta_embeddings']= " "

for i in range(0,len(df1)):

    # create a sentence
    sentence = Sentence(df1['comment_text'][i])

    # embed the sentence
    transformer_embedding.embed(sentence)

    embedding =sentence.embedding.cpu()

    #save vector as numpy
    embedding = embedding.detach().numpy()
    
    # #save vector as pandas dataframe
    embedding = pd.DataFrame(embedding)
    
    # #make list out of sentence
    embedding = embedding[0].tolist()

    # #add the embedding vector to the column of stacked embeddings
    df1['roberta_embeddings'][i] = embedding

In [ ]:
model2 = svm.SVC(C=1, kernel='linear', gamma=1)
model2.fit(list(df1['flair_embeddings']),df1['Sub2_Engaging'])


model3 = svm.SVC(C=10, kernel='linear', gamma=1)
model3.fit(list(df1['roberta_embeddings']),df1['Sub2_Engaging'])



In [ ]:


model2_pred = model2.predict(list(df1['flair_embeddings']))
df1['predictions_flair']=model2_pred


model3_pred = model3.predict(list(df1['roberta_embeddings']))
df1['predictions_roberta']=model3_pred




In [ ]:
#Metrics for FLair
# accuracy: (tp + tn) / (p + n)

print('Metrics for FLair\n')
accuracy = accuracy_score(df1['Sub2_Engaging'], df1['predictions_flair'])
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(df1['Sub2_Engaging'], df1['predictions_flair'],average='macro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(df1['Sub2_Engaging'], df1['predictions_flair'],average='macro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
# f1 = f1_score(df1['Sub1_Toxic'], df1['roberta_predictions'])
# print('F1 score: %f' % f1)

if precision+recall > 0:
  f1_score = 2*precision*recall/(precision+recall)
print('F1 score: %f' % f1_score)

In [ ]:
#Metrics for roberta
# accuracy: (tp + tn) / (p + n)
print('Metrics for Roberta\n')
accuracy = accuracy_score(df1['Sub2_Engaging'], df1['predictions_roberta'])
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(df1['Sub2_Engaging'], df1['predictions_roberta'] ,average='macro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(df1['Sub2_Engaging'], df1['predictions_roberta'] ,average='macro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)

# f1 = f1_score(df1['Sub1_Toxic'], df1['roberta_predictions'])
# print('F1 score: %f' % f1)

if precision+recall > 0:
  f1_score = 2*precision*recall/(precision+recall)
print('F1 score: %f' % f1_score)

In [ ]:
#do majority voting -- Ensemble
df1['Ensemble'] = df1[['predictions_LaBse','predictions_flair' ,'predictions_roberta']].mode(axis=1)[0]



In [ ]:
#Metrics for Ensemble
# accuracy: (tp + tn) / (p + n)
print('Metrics for Ensemble\n')
accuracy = accuracy_score(df1['Sub2_Engaging'], df1['Ensemble'])
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(df1['Sub2_Engaging'], df1['Ensemble'] ,average='macro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(df1['Sub2_Engaging'], df1['Ensemble'] ,average='macro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)

# f1 = f1_score(df1['Sub1_Toxic'], df1['roberta_predictions'])
# print('F1 score: %f' % f1)

if precision+recall > 0:
  f1_score = 2*precision*recall/(precision+recall)
print('F1 score: %f' % f1_score)